In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model, preprocessing

df = pd.read_csv('labels.csv')

(10222, 2)

Part 1 - Check data
Part 2 - Input images
Part 3 - Pick a model